# python的代码优化

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## 基础篇

记住"不成熟的代码优化是万恶之源" - Don Knuth

安装配置工具:
```bash
pip install --pre line-profiler
pip install psutil
pip install memory_profiler
```

参考文献:

1. <http://scipy-lectures.github.com/advanced/optimizing/index.html>
2. <http://pynash.org/2013/03/06/timing-and-profiling.html>

给代码计时
----

In [2]:
import time
import timeit

def f(nsec=1.0):
    """让函数睡眠nsec秒."""
    time.sleep(nsec) 
    
start = timeit.default_timer()
f()
elapsed = timeit.default_timer() - start
elapsed

1.0054981708526611

In [3]:
# 现在我们为了方便，我们用装饰器来实现此功能

def process_time(f, *args, **kwargs):
    def func(*args, **kwargs):
        import timeit
        start = timeit.default_timer()
        f(*args, **kwargs)
        print timeit.default_timer() - start
    return func

In [4]:
@process_time
def f1(nsec):
    """让函数睡眠nsec秒."""
    time.sleep(nsec)

In [5]:
f1(3.0)

3.00531601906


In [6]:
# 在ipython notebook里, 我们可以使用内置函数timeit

In [7]:
%timeit f()

1 loop, best of 3: 1 s per loop


### 给脚本里的每一个函数计时

In [8]:
def f2():
    time.sleep(2)

def f3():
    time.sleep(3)

def f5():
    f2()
    f3()
    
%prun f5()

### 检测内存使用情况

In [9]:
%load_ext memory_profiler

In [10]:
%%file foo.py

def foo(n):
    phrase = 'repeat me'
    pmul = phrase * n
    pjoi = ''.join([phrase for x in xrange(n)])
    pinc = ''
    for x in xrange(n):
        pinc += phrase
    del pmul, pjoi, pinc

Writing foo.py


In [11]:
# mprun 需要代码在文件里
# 在ipython的交互界面里并不工作

from foo import foo

%mprun -f foo foo(100000)

('',)


In [12]:
# 但是, memit 可以用作用于交互函数
# 不像mprun，可以给出逐渐的内存使用分析
# memit给出总共的内存使用情况

def gobble(n):
    x = [i*i for i in range(n)]
    
%memit -r 3 gobble(1000000)

peak memory: 141.58 MiB, increment: 58.23 MiB


## 尝试可能的优化顺序

- 合理的数据结构（Appropriate data structures）
- 合理的算法（Appropriate algorithms）
- 缓存化，记忆化和动态规划（Caching, memoization and dynamic programming）
- 使用python的语言风格（Python-specific idioms）
- 向量化（Vectorization）
- 使用本机代码 (JIT, Cython, wrappers)
- 简单并行化 (IPython parallel, multiprocessing)
- 使用MPI
- 大规模并行编程(Massively parallel programming)
- 使用Map-reduce
- 使用云计算资源(cloud computing resoruces)

## 合理的数据结构

In [14]:
"""查找xs和ys中的独特共有元素. 展示set数据结构的使用."""

def common1(xs, ys):
    """使用lists."""
    zs = set([])
    for x in xs:
        for y in ys:
            if x==y:
                zs.add(x)
    return zs

def common2(xs, ys):
    """使用sets."""
    return set(xs) & set(ys)

def main():
    n = 5000
    xs = npr.randint(1, 100, n)
    ys = npr.randint(50, 150, n)
    
    z1 = common1(xs, ys)
    z2 = common2(xs, ys)
    
    assert(z1 == z2)

xs = np.random.randint(0, 1000, 100)
ys = np.random.randint(0, 1000, 100)
%timeit common1(xs, ys)
%timeit common2(xs, ys)

1000 loops, best of 3: 852 µs per loop
100000 loops, best of 3: 17.7 µs per loop


In [15]:
"""从字典的关键字中查找值. 展现在字典查找中的细微错误."""

adict = dict(zip(np.arange(100000), np.random.randint(0,10,10000)))

def search1(adict, targets):
    """使用一个list."""
    return [adict[t] for t in targets if t in adict.keys()]

def search2(adict, targets):
    """使用一个dict."""
    return [adict[t] for t in targets if t in adict]

targets = np.random.randint(0, 1000000, 10000)
%timeit search1(adict, targets)
%timeit search2(adict, targets)

1 loop, best of 3: 2.19 s per loop
1000 loops, best of 3: 817 µs per loop


In [7]:
"""在每次插入一项的列表中找到最小值. 展示优先队列数据结构的使用."""

from heapq import heappushpop, heapify

def f1(alist, entries):
    """Using repeated sorts."""
    zs = []
    for entry in entries:
        alist.append(entry)
        alist.sort(reverse=True)
        zs.append(alist.pop())
    return zs

def f2(alist, entries):
    """Using a priority queue."""
    heapify(alist)
    zs = []
    for entry in entries:
        zs.append(heappushpop(alist, entry))
    return zs

alist = list(np.random.randint(1000, 100000, 1000))
blist = alist[:]
entries = np.random.randint(1, 10000, 10000)
    
%timeit f1(alist, entries)
%timeit f2(blist, entries)    

1 loop, best of 3: 194 ms per loop
100 loops, best of 3: 2.75 ms per loop


## 合理的算法

In [8]:
"""用穷举搜索和动态规划解决子集和问题."""

import itertools

def knapsack1(size, values):
    """使用穷举搜索."""
    n = len(values)
    if size > np.sum(values):
        return False
    for i in range(1, n):
        for group in itertools.combinations(values, i):
            if size == np.sum(group):
                return True
    return False
            
def knapsack2(size, values):
    """使用动态规划. 注意复杂度的权衡."""
    n = len(values)
    s = np.sum(values)
    if size > np.sum(values):
        return False
    solns = {}
    table = np.zeros((n, s+1), dtype='bool')
    table[0][values[0]] = True
    for i in xrange(1, n):
        for j in xrange(0, s):
            table[i][j] = values[i] == j or table[i-1][j] or table[i-1][j-values[i]]
    return table[-1][size]
    
s = 149
values = np.array([12, 15, 8, 13, 11, 19, 12, 13, 14, 15, 7, 1, 12, 8, 9])
%timeit knapsack1(s, values)
%timeit knapsack2(s, values)

1 loop, best of 3: 209 ms per loop
100 loops, best of 3: 2.02 ms per loop


## 使用python的语言风格

In [9]:
"""字符串的拼接"""

def concat1(alist):
    """使用字符串加法."""
    s = alist[0]
    for item in alist[1:]:
        s += " " + item
    return s
    
def concat2(alist):
    """使用join函数."""
    return " ".join(alist)

alist = ['abcde'] * 1000000
%timeit concat1(alist)
%timeit concat2(alist)    

1 loop, best of 3: 146 ms per loop
100 loops, best of 3: 12.9 ms per loop


In [18]:
"""避免使用循环."""

import math

def loop1(n):
    """使用循环调用方程."""
    z = []
    for i in xrange(n):
        z.append(math.sin(i))
    return z

def loop2(n):
    """使用自带库的方程."""
    z = []
    sin = math.sin
    for i in xrange(n):
        z.append(sin(i))
    return z

def loop3(n):
    """使用列表调用."""
    sin = math.sin
    return [sin(i) for i in xrange(n)]

def loop4(n):
    """使用map方法."""
    sin = math.sin
    return map(sin, xrange(n))

def loop5(n):
    """使用numpy."""
    return np.sin(np.arange(n)).tolist()

n = 1000000
%timeit loop1(n)
%timeit loop2(n)
%timeit loop3(n)
%timeit loop4(n)
%timeit loop5(n)

1 loops, best of 3: 396 ms per loop
1 loops, best of 3: 292 ms per loop
10 loops, best of 3: 192 ms per loop
10 loops, best of 3: 153 ms per loop
10 loops, best of 3: 80.5 ms per loop


In [19]:
"""I/O读写限制（I/O bounded）."""

def io1(xs):
    """使用循环来写入."""
    with open('foo1.txt', 'w') as f:
        for x in xs:
            f.write('%d\t' % x)
    
def io2(xs):
    """写入前使用Join函数."""
    with open('foo2.txt', 'w') as f:
        f.write('\t'.join(map(str, xs)))
        
def io3(xs):
    """Numpy savetxt令人惊讶的超级慢."""
    np.savetxt('foo3.txt', xs, delimiter='\t')
        
def io4(xs):
    """Numpy save好点，如果我们使用二进制形式的话."""
    np.save('foo4.npy', xs)
   
def io5(xs):
    """使用HDF5."""
    import h5py
    with h5py.File("mytestfile.h5", "w") as f:
        ds = f.create_dataset("xs", (len(xs),), dtype='i')
        ds[:] = xs
    
n = 1000*1000
xs = range(n)
%timeit io1(xs)
%timeit io2(xs)
%timeit io3(xs)
%timeit io4(xs)
%timeit io5(xs)

1 loops, best of 3: 1.68 s per loop
1 loops, best of 3: 334 ms per loop
1 loops, best of 3: 6.12 s per loop
10 loops, best of 3: 212 ms per loop
1 loops, best of 3: 160 ms per loop


In [20]:
"""使用缓存区，用空间换时间."""

def fib(n):
    if n <= 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def memoize1(f):
    store = {}
    def func(n):
        if n not in store:
            store[n] = f(n)
        return store[n]
    return func

   
# From http://code.activestate.com/recipes/578231-probably-the-fastest-memoization-decorator-in-the-/

def memoize2(f):
  class memodict(dict):
      __slots__ = ()
      def __missing__(self, key):
          self[key] = ret = f(key)
          return ret
  return memodict().__getitem__
    
# 使用LRU缓存算法来避免过度的内存占用
from pylru import lrudecorator

lrufib = lrudecorator(100)(fib)

@memoize1
def cfib(n):
    return fib(n)

@memoize2
def mfib(n):
    return fib(n)

print fib(10)
print cfib(10)
print mfib(10)
print lrufib(10)

%timeit -n 3 fib(30)
%timeit -n 10 cfib(30)
%timeit -n 10 mfib(30)
%timeit -n 10 lrufib(30)

55
55
55
55
3 loops, best of 3: 372 ms per loop
10 loops, best of 3: 286 ns per loop
10 loops, best of 3: 191 ns per loop
10 loops, best of 3: 2.69 µs per loop


In [21]:
# 我们可以完全的不用递归
# 这是最有效率的方法
# 特别是对比较大的n

def ifib(n):
    t = [0, 1]
    for i in xrange(n):
        t.append(t[-1] + t[-2])
    return t[-2]

%timeit -n 10 ifib(1000)
ifib(10)

10 loops, best of 3: 310 µs per loop


55

In [22]:
# 当然, 最快的办法还是使用闭形式

def binetfib(n):
    sqrt5 = 5**0.5
    Phi = 0.5*(1+sqrt5)
    _phi = 0.5*(1-sqrt5)
    return int((Phi**n  - _phi**n)/sqrt5 + 0.5) # round to correct floating point errors

%timeit -n 10 ifib(1000)
%timeit -n 10 binetfib(1000)
binetfib(10)

10 loops, best of 3: 308 µs per loop
10 loops, best of 3: 1.6 µs per loop


55

## 使用Numpy and Scipy

In [23]:
"""当地操作比拷贝更快速."""

a = np.arange(1e6)
    
%timeit global a; a = a * 0
%timeit global a; a *= 0

100 loops, best of 3: 4.9 ms per loop
100 loops, best of 3: 2.6 ms per loop


In [7]:
"""使用小步进最大化缓存."""

def f1(xs):
    return xs.sum(axis=1)

def f2(xs):
    return xs.sum(axis=0)

xs = np.zeros((1e4, 1e4), order='C') #翻译者注释：order:C代表与C语言类似，行优先；F代表与Fortran类似，列优先
%timeit global xs; f1(xs)
%timeit global xs; f2(xs)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


The slowest run took 9.37 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 48.5 ms per loop
10 loops, best of 3: 73.4 ms per loop


In [25]:
"""使用Numpy indexing来避免循环."""

def idx1(xs):
    """使用循环."""
    s = 0
    for x in xs:
        if (x > 10) and (x < 20):
            s += x
    return s

def idx2(xs):
    """使用索引."""
    return np.sum(xs[(xs > 10) & (xs < 20)])

n = 1000000
xs = np.random.randint(0, 100, n)
%timeit idx1(xs)
%timeit idx2(xs)

1 loops, best of 3: 2.11 s per loop
100 loops, best of 3: 12 ms per loop


In [26]:
"""更多的检索小技巧 - 使用模版操作来计算九个相邻元素的平均值，不使用任何边界条件"""

def average1(xs):
    """使用循环."""
    ys = xs.copy()
    rows, cols = xs.shape
    for i in range(rows):
        for j in range(cols):
            s = 0
            for u in range(i-1, i+2):
                if u < 0 or u >= rows:
                    continue
                for v in range(j-1, j+2):
                    if v < 0 or v >= cols:
                        continue
                    s += xs[u, v]
            ys[i, j] = s/9.0
    return ys

def average2(xs):
    """使用偏移的数列来避免边界检查."""
    rows, cols = xs.shape
    xs1 = np.zeros((rows+2, cols+2))
    xs1[1:-1, 1:-1] = xs[:]
    ys = (xs1[:-2, :-2]  + xs1[1:-1, :-2]  + xs1[2:, :-2] +
          xs1[:-2, 1:-1] + xs1[1:-1, 1:-1] + xs1[2:, 1:-1] +
          xs1[:-2, 2:]   + xs1[1:-1, 2:]   + xs1[2:, 2:])/9.0
    return ys

n = 25
xs = np.random.uniform(0,10,(n, n))
%timeit average1(xs)
%timeit average2(xs)

100 loops, best of 3: 9.32 ms per loop
10000 loops, best of 3: 118 µs per loop


In [27]:
"""使用Repeat和tile函数."""

def main():
    x = np.arange(1,5)
    print x
    print '='*30
    print x.repeat(2)
    print '='*30
    print x.repeat([1,2,3,4])
    print
    print

    y = np.arange(1,9).reshape(2,4)
    print y
    print '='*30
    print y.repeat(2)
    print '='*30
    print y.repeat(2, axis=1)
    print '='*30
    print y.repeat(2, axis=0)
    print '='*30
    print y.repeat([1,2,3,4], axis=1)
    print '='*30
    print y.repeat([1,2], axis=0)
    print 
    print

    z = np.arange(1, 5).reshape(2,2)
    print z
    print '='*30
    # make 1 row and 2 cols of the block z
    # broadcasting rules prepend1 to missing dimensions
    # so tile(z, 2) is equivalnet to tile(z, (1,2))
    print np.tile(z, 2)
    print '='*30
    # make 2 rows and 3 cols of the block z
    print np.tile(z, (2,3))
    
main()

[1 2 3 4]
[1 1 2 2 3 3 4 4]
[1 2 2 3 3 3 4 4 4 4]


[[1 2 3 4]
 [5 6 7 8]]
[1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8]
[[1 1 2 2 3 3 4 4]
 [5 5 6 6 7 7 8 8]]
[[1 2 3 4]
 [1 2 3 4]
 [5 6 7 8]
 [5 6 7 8]]
[[1 2 2 3 3 3 4 4 4 4]
 [5 6 6 7 7 7 8 8 8 8]]
[[1 2 3 4]
 [5 6 7 8]
 [5 6 7 8]]


[[1 2]
 [3 4]]
[[1 2 1 2]
 [3 4 3 4]]
[[1 2 1 2 1 2]
 [3 4 3 4 3 4]
 [1 2 1 2 1 2]
 [3 4 3 4 3 4]]


In [28]:
"""使用Random choice函数."""

def main():
    x = np.arange(1, 30)
    k = 10

    # 带放回的抽取十项 
    print np.random.choice(x, 10)

    # 不带放回的抽取十项
    print np.random.choice(x, 10, replace=False)

    # 使用加权抽取，更容易抽到大数
    wts = np.arange(1, len(x)+1, dtype='float')
    wts /= np.sum(wts)
    print np.random.choice(x, 10, p=wts)
    
main()

[ 8  3  6 23  8 14 15 18 26 23]
[25 13  8 17 18  3 24  9 21 15]
[21 10 22 10 28 28 21  4 14 23]


In [29]:
"""距离计算."""

from scipy.spatial.distance import pdist, cdist, squareform

def main():
    # find all pairwise distance between a set of points
    xs = np.arange(1, 13).reshape(4, 3)
    print squareform(pdist(xs), 'sqeuclidean')
    print '=' * 30

    # find all pairwise distances between 2 sets of points
    print cdist(xs, xs[::-1], 'sqeuclidean')
    
main()

[[  0.           5.19615242  10.39230485  15.58845727]
 [  5.19615242   0.           5.19615242  10.39230485]
 [ 10.39230485   5.19615242   0.           5.19615242]
 [ 15.58845727  10.39230485   5.19615242   0.        ]]
[[ 243.  108.   27.    0.]
 [ 108.   27.    0.   27.]
 [  27.    0.   27.  108.]
 [   0.   27.  108.  243.]]


In [30]:
"""向量和矩阵的配对内积."""

# 找到向量的配对内积
from numpy.core.umath_tests import inner1d

def main():
    xs = np.arange(1,13).reshape(3,4)
    print inner1d(xs, xs[::-1])

    print '='*30

    # 找到矩阵的配对内积
    from numpy.core.umath_tests import matrix_multiply

    xs = np.random.randint(0, 10, (5, 2, 2))
    ys = np.random.randint(0, 10, (5, 2, 2))
    print matrix_multiply(xs, ys)
    
main()

[110 174 110]
[[[65 72]
  [81 81]]

 [[26 67]
  [20 71]]

 [[57 36]
  [48 63]]

 [[48 64]
  [38 36]]

 [[63 45]
  [28 88]]]


In [31]:
"""使用存储器映射来从文件中分块读取数据."""

from tempfile import mktemp

def main():
    s = 6
    x = np.arange(s*s, dtype='float').reshape(s, s)
    f = mktemp()
    x.tofile(f)

    y1 = np.memmap(f, dtype='float', offset=0, shape=(s/2, s))
    y2 = np.memmap(f, dtype='float', offset=s/2*x[0].nbytes, shape=(s/2, s))

    print x
    print '='*30
    print y1
    print '='*30
    print y2
    
main()

[[  0.   1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.  11.]
 [ 12.  13.  14.  15.  16.  17.]
 [ 18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.]
 [ 30.  31.  32.  33.  34.  35.]]
[[  0.   1.   2.   3.   4.   5.]
 [  6.   7.   8.   9.  10.  11.]
 [ 12.  13.  14.  15.  16.  17.]]
[[ 18.  19.  20.  21.  22.  23.]
 [ 24.  25.  26.  27.  28.  29.]
 [ 30.  31.  32.  33.  34.  35.]]


In [32]:
"""向量化一个方程."""

def tlog(x):
    if x < 1:
        return 0
    else:
        return np.log(x)
    
vtlog = np.vectorize(tlog, otypes='d')

def func1(xs):
    return np.array(map(tlog, xs))

def func2(xs):
    return vtlog(xs)

n = 100000
xs = np.random.uniform(-10, 10, n)
%timeit func1(xs)
%timeit func2(xs)

1 loops, best of 3: 433 ms per loop
1 loops, best of 3: 216 ms per loop


In [33]:
"""使用numexpr. numexpr能够提供自动的并行化处理和更智能的JIT(just in time)编译到C语言."""

import numexpr as ne

a = np.random.rand(1e6)

%timeit global a; np.sin(a)**2 + np.cos(a)**2
%timeit global a; ne.evaluate("sin(a)**2 + cos(a)**2")

import numba
@numba.autojit
def f(a):
    return np.sin(a)**2 + np.cos(a)**2

%timeit f(a)

10 loops, best of 3: 63.8 ms per loop
100 loops, best of 3: 7.8 ms per loop
1 loops, best of 3: 65.7 ms per loop


## 并行化

In [34]:
"""基本多线程应用的例子"""

def slow_calc(*args):
    import time
    time.sleep(1)
    return args

def serial(njobs):
    results = []
    for i in range(njobs):
        results.append(slow_calc([i]))
    return results
       
def parallel(njobs):
    from multiprocessing import Pool
    p = Pool()
    
    argss = [[i] for i in range(njobs)]
    p.map(slow_calc, argss)
    
njobs = 2
%timeit serial(njobs)
%timeit parallel(njobs)

1 loops, best of 3: 2 s per loop
1 loops, best of 3: 1.03 s per loop


In [14]:
"""比较同步版和异步的apply和map函数. Python里也有一个threading模块, 但是对于数值计算, 
mutliprocessing模块更佳直截了当."""

def calc(args):
    i, j = args
    return i, j

def calc2(i, j):
    return i, j

def main():
    n = 5
    
    from multiprocessing import Pool
    argss = [(i, i) for i in range(n)]
    p = Pool()

    """使用同步的map函数."""
    results1 = p.map(calc, argss)
    # blocks until all workers complete their tasks
    print results1
    
    """使用异步的map函数."""
    results2 = p.map_async(calc, argss)
    # Do other stuff here
    print results2.get()
    
    """ 使用同步的apply函数."""
    results3 = []
    for i in range(n):
        args = (i, i)
        results3.append(p.apply(calc2, args))
    # blocks until all workers complete their tasks
    print results3
    
    """ 使用异步的apply函数."""
    results4 = []
    for i in range(n):
        args = (i, i)
        results4.append(p.apply_async(calc2, args))
    # Do other stuff here
    print [worker.get() for worker in results4]
    
main()

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]


In [36]:
"""multiprocessing.Pool的内存溢出问题.
默认设置下，每个任务完成后并不会释放内存，只有等所有任务都完成后才会释放内存.
这个设置能通过调整参数maxtasksperchild来改变.
"""

from multiprocessing import Pool

# each thread is destroyed and recreated after every task
# use only 4 threadss.
p = Pool(4, maxtasksperchild=1)

In [37]:
"""IPython里的交互式并行化. 

Python支持交互式的并行处理. 细节请看

http://ipython.org/ipython-doc/dev/parallel/parallel_intro.html#parallel-overview and 
http://minrk.github.com/scipy-tutorial-2011/

."""

pass

## Cython

In [38]:
"""接下来一系列例子展示Cython是如何更加快速的运行代码的."""
import operator

def adder1(xs):
    """展示性Python代码."""
    sqrt = np.sqrt
    add = operator.add
    s = reduce(add, map(sqrt, xs), 0)
    return s

In [15]:
%load_ext cythonmagic

//anaconda/lib/python2.7/site-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")


In [40]:
%%cython
cimport numpy as np
cimport cython

# 从C的库载入方程
cdef extern from "math.h":
    double sqrt(double)

# 这是一个慢的例子 - 下个例子将告诉你如何让它变得更快
cdef double cadder1(xs):
    cdef double s = 0
    cdef int i
    cdef int n = len(xs)
    for i in range(n):
        s += sqrt(xs[i])
    return s

def adder2(xs not None):
    return cadder1(xs)

# [1] 使用numpy array的数据类型，使他变得更快
cdef double cadder2(np.ndarray[np.double_t, ndim=1] xs):
    cdef double s = 0
    cdef unsigned int i # [2] 关闭负索引的检查
    cdef int n = len(xs)
    for i in range(n):
        s += sqrt(xs[i])
    return s

# [3] 关闭装饰器的边界检查
@cython.boundscheck(False)
def adder3(xs not None):
    return cadder2(xs)

In [41]:
def main():
    import time
    
    n = 1e6
    xs = np.random.rand(n)
    
    start = time.time()
    z1 = adder1(xs)
    print "'adder1' %.03f sec" % (time.time() - start)
    
    start = time.time()
    z2 = adder2(xs)
    print "'adder2' %.03f sec" % (time.time() - start)
        
    start = time.time()
    z3 = adder3(xs)
    print "'adder3' %.03f sec" % (time.time() - start)
            
    # 使用numpy作为比较
    start = time.time()
    z4 = np.sqrt(xs).sum()
    print "'adder4' %.03f sec" % (time.time() - start)
    
    assert(z1 == z2 == z3 == z4)

main()

'adder1' 3.200 sec
'adder2' 0.143 sec
'adder3' 0.007 sec
'adder4' 0.010 sec


## 参考文献

In [42]:
%load_ext version_information

%version_information numpy, numexpr, numba

Software versions
Python 2.7.7 |Anaconda 2.0.0 (x86_64)| (default, Jun  2 2014, 12:48:16) [GCC 4.0.1 (Apple Inc. build 5493)]
IPython 2.1.0
OS posix [darwin]
numpy 1.8.1
numexpr 2.3.1
numba 0.13.1
<tr><td colspan='2'>Mon Jun 09 10:32:33 2014 EDT</td></tr>